In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [2]:
# Define dataset paths
train_dir = "sign_data/train/"
train_csv = "sign_data/train_data.csv"
test_dir = "sign_data/test/"
test_csv = "sign_data/test_data.csv"

In [3]:
# Load dataset
df_train = pd.read_csv(train_csv)
df_test = pd.read_csv(test_csv)

In [4]:
# Image Preprocessing Function
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Ensure grayscale
    img = cv2.resize(img, (100, 100))  # Resize to match model input
    img = img / 255.0  # Normalize pixel values
    return img

In [5]:
# Load training data
train_images1, train_images2, train_labels = [], [], []
for j in range(len(df_train)):
    img1 = load_and_preprocess_image(train_dir + df_train.iat[j, 0])
    img2 = load_and_preprocess_image(train_dir + df_train.iat[j, 1])
    train_images1.append(img1)
    train_images2.append(img2)
    train_labels.append(df_train.iat[j, 2])  # Assuming 0 or 1 labels

# Convert training data to numpy arrays
train_images1 = np.array(train_images1).reshape(-1, 100, 100, 1)  # Ensure shape consistency
train_images2 = np.array(train_images2).reshape(-1, 100, 100, 1)
train_labels = np.array(train_labels).astype(int)  # Ensure integer labels

In [6]:
# Convert labels to one-hot encoding if using categorical crossentropy
train_labels = to_categorical(train_labels, num_classes=2)

In [7]:
# Load test data
test_images1, test_images2, test_labels = [], [], []
for j in range(len(df_test)):
    img1 = load_and_preprocess_image(test_dir + df_test.iat[j, 0])
    img2 = load_and_preprocess_image(test_dir + df_test.iat[j, 1])
    test_images1.append(img1)
    test_images2.append(img2)
    test_labels.append(df_test.iat[j, 2])

# Convert test data to numpy arrays
test_images1 = np.array(test_images1).reshape(-1, 100, 100, 1)
test_images2 = np.array(test_images2).reshape(-1, 100, 100, 1)
test_labels = np.array(test_labels).astype(int)
test_labels = to_categorical(test_labels, num_classes=2)

In [12]:
# Define the Siamese Network
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def create_siamese_network(input_shape):
    input_1 = Input(input_shape)
    input_2 = Input(input_shape)
    
    shared_dense = Flatten()(input_1)
    shared_dense = Dense(128, activation="relu")(shared_dense)
    shared_dense = Dense(64, activation="relu")(shared_dense)
    
    shared_dense_2 = Flatten()(input_2)
    shared_dense_2 = Dense(128, activation="relu")(shared_dense_2)
    shared_dense_2 = Dense(64, activation="relu")(shared_dense_2)

    distance = Lambda(euclidean_distance)([shared_dense, shared_dense_2])
    output = Dense(2, activation="softmax")(distance)

    return Model(inputs=[input_1, input_2], outputs=output)

In [9]:
# Create and compile the model
input_shape = (100, 100, 1)
model = create_siamese_network(input_shape)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.00006), metrics=["accuracy"])

# Train the model
history = model.fit([train_images1, train_images2], train_labels, validation_split=0.30,
      batch_size=32, verbose=1, epochs=20)


Epoch 1/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 23s 37ms/step - accuracy: 0.5609 - loss: 1.0325 - val_accuracy: 0.4984 - val_loss: 0.7985
Epoch 2/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 15s 30ms/step - accuracy: 0.5645 - loss: 0.7453 - val_accuracy: 0.4984 - val_loss: 0.8380
Epoch 3/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.5549 - loss: 0.7335 - val_accuracy: 0.4984 - val_loss: 0.9338
Epoch 4/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.5661 - loss: 0.7252 - val_accuracy: 0.4984 - val_loss: 0.8018
Epoch 5/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.5619 - loss: 0.6930 - val_accuracy: 0.4984 - val_loss: 0.7484
Epoch 6/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.5654 - loss: 0.6512 - val_accuracy: 0.4983 - val_loss: 0.7550
Epoch 7/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - accuracy: 0.5599 - loss: 0.6345 - val_accuracy: 0.4984 - val_loss: 0.8321
Epoch 8/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 18s 34ms/step - accuracy: 0.5628 - loss: 0.6257 -

In [10]:
model.save("siamese_model.h5") 

In [ ]:
from tensorflow.keras.models import load_model

# Load model from HDF5 file
model = load_model("siamese_model.h5")

# Or load from SavedModel format
model = load_model("siamese_model")


In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

# Define the Euclidean distance function again
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

# Load model with custom_objects
model = load_model("siamese_model.h5", custom_objects={"euclidean_distance": euclidean_distance})
